In [22]:
import os
import re
import nltk
from nltk import word_tokenize , sent_tokenize , pos_tag
from tdmh import *
import pandas as pd
import ja_core_news_trf

In [18]:
!pip install -U pip setuptools wheel
!pip install -U spacy

     |████████████████████████████████| 2.1 MB 3.1 MB/s eta 0:00:01     |███████████████████████████████ | 2.1 MB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 4.8 MB/s eta 0:00:01
  Attempting uninstall: wheel
    Found existing installation: wheel 0.37.0
    Uninstalling wheel-0.37.0:
      Successfully uninstalled wheel-0.37.0
  Attempting uninstall: setuptools
    Found existing installation: setuptools 58.0.4
    Uninstalling setuptools-58.0.4:
      Successfully uninstalled setuptools-58.0.4
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
conda-repo-cli 1.0.4 requires pat

In [2]:
!python -m spacy download ja_core_news_trf
!python -m spacy download ja_core_news_sm
!python -m spacy download ja_core_news_md
!python -m spacy download ja_core_news_lg

  Using cached https://github.com/explosion/spacy-models/releases/download/ja_core_news_trf-3.2.0/ja_core_news_trf-3.2.0-py3-none-any.whl (337.9 MB)


✔ Download and installation successful
You can now load the package via spacy.load('ja_core_news_trf')
  Using cached https://github.com/explosion/spacy-models/releases/download/ja_core_news_sm-3.2.0/ja_core_news_sm-3.2.0-py3-none-any.whl (13.0 MB)
✔ Download and installation successful
You can now load the package via spacy.load('ja_core_news_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 MB 4.6 MB/s eta 0:00:0000:0100:01


✔ Download and installation successful
You can now load the package via spacy.load('ja_core_news_md')
     ━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/556.2 MB 10.1 MB/s eta 0:00:47
ERROR: Wheel 'ja-core-news-lg' located at /private/var/folders/hb/v9_vkgfd41j65wtt0vnrlt2h0000gn/T/pip-unpack-9_1kazf2/ja_core_news_lg-3.2.0-py3-none-any.whl is invalid.


# Organise the tweets

Read all the downloaded tweets from the TSV files.

In [4]:
path = os.path.join( 'HanaKimura' , 'tweets_hanakimura.tsv' )
df1 = pd.read_csv( path  , encoding = 'utf-8' , sep = '\t' )

path = os.path.join( 'HanaKimura' , 'tweets_hanakimura_japanese.tsv' )
df2 = pd.read_csv( path  , encoding = 'utf-8' , sep = '\t' )

path = os.path.join( 'HanaKimura' , 'tweets_kimurahana.tsv' )
df3 = pd.read_csv( path  , encoding = 'utf-8' , sep = '\t' )

Change the 'created_at' date into a 'dateframe'

In [5]:
df = pd.concat([df1, df2, df3])
df['created_at']  = pd.to_datetime( df['created_at'] ).dt.tz_localize(None)

The date of Hana Kimura's death

In [6]:
date = pd.to_datetime( '2020-05-23' )

In [7]:
dir = 'Corpus'

if not os.path.exists(dir):
    os.mkdir(dir)

path = os.path.join(dir, 'tweets_before_japanese.txt')
out_before_japanese = open(  path, 'w' , encoding = 'utf-8')

path = os.path.join(dir, 'tweets_before_english.txt' )
out_before_english = open( path , 'w' , encoding = 'utf-8')

path = os.path.join(dir, 'tweets_after_japanese.txt' )
out_after_japanese = open( path , 'w' , encoding = 'utf-8')

path = os.path.join(dir, 'tweets_after__english.txt' )
out_after_english = open( path , 'w' , encoding = 'utf-8')


for i,row in df.iterrows():
    if not re.search( r'^RT' , row['text'] ):
        if row['created_at'] < date:
            if row['lang'] == 'ja' :
                out_before_japanese.write( f"{row['text']}\n" )
            else:
                out_before_english.write( f"{row['text']}\n" )
        else:
            if row['lang'] == 'ja':
                out_after_japanese.write( f"{row['text']}\n" )
            else:
                out_after_english.write( f"{row['text']}\n" )
            
        
out_before_japanese.close()
out_before_english.close()
out_after_japanese.close()
out_after_english.close()

## Create a list of all the files in the corpus

In [8]:
texts = []
dir = 'Corpus'

for file in os.listdir(dir):
    if re.search( r'txt$' , file ):
        path = os.path.join( dir , file )
        texts.append(path)

## create text analysis functions

In [14]:
def get_all_hashtags(full_text):
    freq = dict()
    hash_tags = re.findall( r'#\w+\b' , full_text )
    for tag in hash_tags:
        freq[tag] = freq.get(tag,0) + 1
    return freq

def merge_dictionaries(x,y):
    z = x.copy()   
    z.update(y)   
    return z


def get_title(path):
    title = os.path.basename(path)
    if re.search( r'txt$' , title ):
        # Remove txt extension
        title = title[ :title.index('.txt') ]
        # remove commas and dots
        title = re.sub( r'[.,]' , '' , title )
    return title

## create data for all  the texts in the corpus

In [12]:
all_tags = dict()

for text in texts: 
    f = open( text , encoding = 'utf-8')
    full_text = f.read()
    hash_tags = get_all_hashtags(full_text)
    all_tags = merge_dictionaries(all_tags,hash_tags)

most_frequent = []
for tag in sortedByValue( all_tags , ascending = False):
    if all_tags[tag] > 200:
        most_frequent.append(tag)
        
print(most_frequent)

['#誹謗中傷', '#木村花', '#スターダム', '#文春オンライン', '#フジテレビ', '#RIPHanaKimura', '#週刊文春', '#スクープ速報', '#TCS', '#テラスハウス', '#自殺', '#stardom', '#テラハ', '#SNS']


In [13]:
out = open( 'data.csv' , 'w' , encoding = 'utf-8' )

out.write('title')
for t in most_frequent:
    out.write( f',{t}' )
out.write('\n')

for text in texts:
    
    data = dict()
    print( f'Analysing {text} ...')
    
    ## Get the title, based on the filename
    title = get_title( text )
    
    ## read the full text
    fh = open( text, encoding = 'utf-8')
    full_text = fh.read()
    
    hash_tags = get_all_hashtags(full_text)
    
    out.write( f'{title}')
    for t in most_frequent:
        out.write( f',{hash_tags.get(t,0)}' )
    out.write('\n')
    
out.close()
print('Done!')

Analysing Corpus/tweets_before_english.txt ...
Analysing Corpus/tweets_after__english.txt ...
Analysing Corpus/tweets_after_japanese.txt ...
Analysing Corpus/tweets_before_japanese.txt ...
Done!


In [23]:
import pandas as pd

df = pd.read_csv('data.csv')

In [24]:
df.head()

,title,#誹謗中傷,#木村花,#スターダム,#文春オンライン,#フジテレビ,#RIPHanaKimura,#週刊文春,#スクープ速報,#TCS,#テラスハウス,#自殺,#stardom,#テラハ,#SNS
0,tweets_before_english,0,310,220,0,0,0,0,0,42,14,0,20,2,0
1,tweets_after__english,60,1330,114,0,12,956,3,0,23,122,10,80,21,9
2,tweets_after_japanese,2623,19807,1144,886,693,598,586,477,81,5281,240,185,1313,220
3,tweets_before_japanese,0,1911,1039,0,0,0,0,0,464,286,0,236,223,0


In [25]:
df.shape

(4, 15)

In [26]:
df['#誹謗中傷']

0       0
1      60
2    2623
3       0
Name: #誹謗中傷, dtype: int64